In [ ]:
import pandas as pd
#cargar csv imagenes
df= pd.read_csv("/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay.csv")
#reemplazar la palabra POST por IMG en columnas Imagen y Ruta
df["Imagen"] = df["Imagen"].str.replace("POST", "IMG")
df["Ruta"] = df["Ruta"].str.replace("POST", "IMG")
#comprobar
print(df.head())

    id_post       Fecha       Imagen  \
0   POST_98  2024-08-05   IMG_98.jpg   
1  POST_190  2024-08-08  IMG_190.jpg   
2   POST_38  2024-08-12   IMG_38.jpg   
3   POST_73  2024-08-14   IMG_73.jpg   
4   POST_44  2024-08-16   IMG_44.jpg   

                                                Ruta  
0  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
1  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
2  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
3  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  
4  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...  


In [6]:
#descargar nuevo df resultante con el nombre publicaciones_pixabay_ok.csv
df.to_csv("/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv", index=False)
print("Archivo actualizado y descargado correctamente.")

Archivo actualizado y descargado correctamente.


In [9]:
import pandas as pd
import os

print("🔍 SOLUCIÓN AL PROBLEMA DE SEPARADORES")
print("=" * 50)

# Cargar archivos con separadores correctos
try:
    print("📂 Cargando data_demo.csv...")
    df_demo = pd.read_csv('/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/data_demo.csv', sep=';')
    print(f"✅ data_demo_ok cargado: {len(df_demo)} filas, {len(df_demo.columns)} columnas")
    print("📋 Columnas:", df_demo.columns.tolist())
    
except Exception as e:
    print(f"❌ Error: {e}")

try:
    print("\n📂 Cargando publicaciones_pixabay_ok.csv...")
    df_pixabay = pd.read_csv('/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv', sep=',')
    print(f"✅ publicaciones_pixabay_ok cargado: {len(df_pixabay)} filas, {len(df_pixabay.columns)} columnas")
    print("📋 Columnas:", df_pixabay.columns.tolist())
    
except Exception as e:
    print(f"❌ Error: {e}")

# Limpiar nombres de columnas por si acaso
df_demo.columns = df_demo.columns.str.strip()
df_pixabay.columns = df_pixabay.columns.str.strip()

# Verificar columnas id_post
print(f"\n🔍 VERIFICACIÓN:")
print(f"¿Existe 'id_post' en df_demo? {'id_post' in df_demo.columns}")
print(f"¿Existe 'id_post' en df_pixabay? {'id_post' in df_pixabay.columns}")

# Identificar columnas comunes antes del merge
common_columns = set(df_demo.columns) & set(df_pixabay.columns)
print(f"📊 Columnas comunes: {list(common_columns)}")

if 'id_post' in df_demo.columns and 'id_post' in df_pixabay.columns:
    print("\n✅ ¡AMBAS COLUMNAS ENCONTRADAS!")
    
    # Mostrar algunos valores para verificar compatibilidad
    print(f"Valores de ejemplo en demo: {df_demo['id_post'].head().tolist()}")
    print(f"Valores de ejemplo en pixabay: {df_pixabay['id_post'].head().tolist()}")
    
    # Verificar si hay coincidencias antes del merge
    common_ids = set(df_demo['id_post']) & set(df_pixabay['id_post'])
    print(f"📊 IDs comunes encontrados: {len(common_ids)}")
    if len(common_ids) > 0:
        print(f"Ejemplos de IDs comunes: {list(common_ids)[:5]}")
    else:
        print("⚠️ No hay IDs comunes entre los archivos")
    
    # Hacer el merge con sufijos explícitos para columnas duplicadas
    print("\n🔗 REALIZANDO MERGE...")
    result = pd.merge(df_demo, df_pixabay, on='id_post', how='left', suffixes=('_demo', '_pixabay'))
    
    print(f"✅ Merge exitoso: {len(result)} filas, {len(result.columns)} columnas")
    
    # Mostrar todas las columnas del resultado para ver qué hay disponible
    print(f"📋 Columnas en el resultado: {result.columns.tolist()}")
    
    # Verificar cuántas filas se mergearon exitosamente
    # Buscar columnas que vengan solo del archivo pixabay
    pixabay_only_cols = []
    for col in df_pixabay.columns:
        if col != 'id_post':  # Excluir la columna de merge
            if col in df_demo.columns:
                # Si la columna existe en ambos, buscar la versión _pixabay
                pixabay_col_name = f"{col}_pixabay"
                if pixabay_col_name in result.columns:
                    pixabay_only_cols.append(pixabay_col_name)
            else:
                # Si la columna solo existe en pixabay
                if col in result.columns:
                    pixabay_only_cols.append(col)
    
    print(f"📋 Columnas de pixabay en resultado: {pixabay_only_cols}")
    
    if pixabay_only_cols:
        merged_count = result[pixabay_only_cols[0]].notna().sum()
        print(f"📊 Filas con datos de pixabay: {merged_count}")
        
        # Buscar la columna de imagen (puede tener diferentes nombres)
        imagen_cols = [col for col in result.columns if 'imagen' in col.lower()]
        if imagen_cols:
            imagen_col = imagen_cols[0]
            imagen_count = result[imagen_col].notna().sum()
            print(f"📊 Filas con imágenes (columna '{imagen_col}'): {imagen_count}")
        else:
            print("⚠️ No se encontró columna de imagen")
    
    # Limpiar el resultado - mantener solo una versión de las columnas duplicadas
    print("\n🧹 LIMPIANDO COLUMNAS DUPLICADAS...")
    result_clean = result.copy()
    
    # Para cada columna común, decidir cuál mantener
    for col in common_columns:
        if col != 'id_post':  # No tocar la columna de merge
            demo_col = f"{col}_demo"
            pixabay_col = f"{col}_pixabay"
            
            if demo_col in result_clean.columns and pixabay_col in result_clean.columns:
                # Combinar: usar pixabay si está disponible, sino demo
                result_clean[col] = result_clean[pixabay_col].fillna(result_clean[demo_col])
                # Eliminar las columnas con sufijos
                result_clean = result_clean.drop([demo_col, pixabay_col], axis=1)
                print(f"✅ Columna '{col}' combinada")
    
    print(f"📋 Columnas después de limpiar: {result_clean.columns.tolist()}")
    
    # Definir la ruta completa para guardar el archivo
    output_path = '/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/data_unificada.csv'
    
    # Verificar que el directorio existe
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"📁 Directorio creado: {output_dir}")
    
    # Guardar resultado limpio en la ruta especificada
    result_clean.to_csv(output_path, index=False, sep=';')
    print(f"✅ Archivo guardado en: {output_path}")
    
    # Verificar que el archivo se guardó correctamente
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path)
        print(f"📁 Archivo verificado: {file_size} bytes")
    else:
        print("❌ Error: El archivo no se guardó correctamente")
    
    # Mostrar algunas filas de ejemplo con columnas disponibles
    print("\n📊 EJEMPLO DEL RESULTADO:")
    try:
        # Intentar mostrar con columnas específicas que deberían existir
        cols_to_show = ['id_post']
        
        # Añadir columnas importantes si existen
        important_cols = ['Fecha', 'Imagen', 'Ruta', 'Canal', 'Formato', 'Alcance']
        for col in important_cols:
            if col in result_clean.columns:
                cols_to_show.append(col)
        
        # Mostrar máximo 6 columnas
        cols_to_show = cols_to_show[:6]
        print(f"Columnas a mostrar: {cols_to_show}")
        print(result_clean[cols_to_show].head())
        
    except Exception as e:
        print(f"Error mostrando ejemplo: {e}")
        print("Mostrando primeras 5 columnas:")
        print(result_clean.iloc[:5, :5])
    
    # Estadísticas finales
    print(f"\n📈 ESTADÍSTICAS DEL MERGE:")
    print(f"- Filas originales en demo: {len(df_demo)}")
    print(f"- Filas en pixabay: {len(df_pixabay)}")
    print(f"- Filas en resultado: {len(result_clean)}")
    
    if pixabay_only_cols:
        # Buscar la primera columna de pixabay en el resultado limpio
        pixabay_col_clean = None
        for col in ['Imagen', 'Ruta']:
            if col in result_clean.columns:
                pixabay_col_clean = col
                break
        
        if pixabay_col_clean:
            merged_rows = result_clean[pixabay_col_clean].notna().sum()
            print(f"- Filas que se mergearon exitosamente: {merged_rows}")
            print(f"- Porcentaje de merge exitoso: {merged_rows/len(result_clean)*100:.1f}%")
    
else:
    print("❌ Aún no se encuentran las columnas")
    print(f"Columnas en demo: {df_demo.columns.tolist()}")
    print(f"Columnas en pixabay: {df_pixabay.columns.tolist()}")

print(f"\n📊 RESUMEN FINAL:")
print(f"data_demo_ok shape: {df_demo.shape}")
print(f"publicaciones_pixabay_ok shape: {df_pixabay.shape}")
if 'result_clean' in locals():
    print(f"Resultado final shape: {result_clean.shape}")
    print(f"📁 Archivo guardado en: /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/data_unificada.csv")
    

🔍 SOLUCIÓN AL PROBLEMA DE SEPARADORES
📂 Cargando data_demo.csv...
✅ data_demo_ok cargado: 200 filas, 24 columnas
📋 Columnas: ['id_post', 'Fecha', 'Hora', 'Canal', 'Formato', 'Imagen', 'Alcance', 'Interacciones', 'Reproducciones', 'Duracion_video', 'DM', 'Retencion', 'Visitas_perfil', 'Link_bio', 'Visitas_producto', 'Carrito', 'Compras', 'Valor_compra', 'Contacto', 'Inversion', 'ROI', 'Engagement', 'CPC', 'Unnamed: 23']

📂 Cargando publicaciones_pixabay_ok.csv...
✅ publicaciones_pixabay_ok cargado: 200 filas, 4 columnas
📋 Columnas: ['id_post', 'Fecha', 'Imagen', 'Ruta']

🔍 VERIFICACIÓN:
¿Existe 'id_post' en df_demo? True
¿Existe 'id_post' en df_pixabay? True
📊 Columnas comunes: ['Fecha', 'id_post', 'Imagen']

✅ ¡AMBAS COLUMNAS ENCONTRADAS!
Valores de ejemplo en demo: ['POST_190', 'POST_146', 'POST_74', 'POST_183', 'POST_198']
Valores de ejemplo en pixabay: ['POST_98', 'POST_190', 'POST_38', 'POST_73', 'POST_44']
📊 IDs comunes encontrados: 131
Ejemplos de IDs comunes: ['POST_125', 'POST_

In [2]:
import pandas as pd
import re

print("🔄 CONVIRTIENDO RUTAS LOCALES A URLs DE GOOGLE DRIVE")
print("=" * 60)

# Cargar el CSV actual
print("📂 Cargando publicaciones_pixabay_ok.csv...")
csv_path = "/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv"
df = pd.read_csv(csv_path)

print(f"✅ CSV cargado: {len(df)} filas")
print(f"📋 Columnas actuales: {df.columns.tolist()}")

# Mostrar algunas rutas actuales
print(f"\n🔍 Ejemplos de rutas actuales:")
print(df[['Imagen', 'Ruta']].head(3))

# Función para extraer el número de imagen y crear URL de Google Drive
def crear_url_google_drive(imagen_name):
    """
    Extrae el número de la imagen (ej: IMG_98.jpg -> 98) 
    y crea una URL de Google Drive usando ese número como ID
    """
    # Extraer número de la imagen (ej: IMG_98.jpg -> 98)
    match = re.search(r'IMG_(\d+)\.jpg', imagen_name)
    if match:
        numero = match.group(1)
        # Por ahora usamos un ID base + número para simular los IDs de Google Drive
        # TÚ DEBERÁS REEMPLAZAR ESTO CON LOS IDs REALES DE TUS ARCHIVOS
        # Ejemplo: si IMG_98.jpg está en Google Drive con ID "145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8"
        # entonces debes mapear 98 -> "145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8"
        
        # Para el ejemplo, uso un mapeo automático (¡ESTO LO DEBES CAMBIAR!)
        google_drive_id = f"1{numero}ABC{numero}XYZ{numero}"  # ID ficticio
        url = f"https://drive.google.com/uc?id={google_drive_id}"
        return url
    return None

# Crear la nueva columna URL_Publica
print(f"\n🔗 Creando URLs de Google Drive...")
df['URL_Publica'] = df['Imagen'].apply(crear_url_google_drive)

print(f"✅ URLs creadas para {df['URL_Publica'].notna().sum()} imágenes")

# Mostrar ejemplos de las nuevas URLs
print(f"\n📋 Ejemplos de URLs generadas:")
print(df[['Imagen', 'Ruta', 'URL_Publica']].head(3))

# Guardar el archivo actualizado
output_path = "/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_urls.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ Archivo guardado en: {output_path}")

print(f"\n⚠️  IMPORTANTE - PASOS SIGUIENTES:")
print(f"1. Sube TODAS tus imágenes (IMG_98.jpg, IMG_190.jpg, etc.) a Google Drive")
print(f"2. Haz cada imagen pública (botón derecho -> Obtener enlace -> Cualquier persona con el enlace)")
print(f"3. Para cada imagen, copia su ID del enlace público")
print(f"4. Actualiza la función crear_url_google_drive() con un mapeo real")
print(f"5. Ejemplo: si IMG_98.jpg tiene ID '145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8':")
print(f"   entonces 98 -> '145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8'")

print(f"\n📖 FORMATO DE URL CORRECTO:")
print(f"Enlace público: https://drive.google.com/file/d/ID_ARCHIVO/view?usp=sharing")
print(f"URL directa:    https://drive.google.com/uc?id=ID_ARCHIVO")

🔄 CONVIRTIENDO RUTAS LOCALES A URLs DE GOOGLE DRIVE
📂 Cargando publicaciones_pixabay_ok.csv...
✅ CSV cargado: 200 filas
📋 Columnas actuales: ['id_post', 'Fecha', 'Imagen', 'Ruta']

🔍 Ejemplos de rutas actuales:
        Imagen                                               Ruta
0   IMG_98.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...
1  IMG_190.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...
2   IMG_38.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...

🔗 Creando URLs de Google Drive...
✅ URLs creadas para 200 imágenes

📋 Ejemplos de URLs generadas:
        Imagen                                               Ruta  \
0   IMG_98.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...   
1  IMG_190.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...   
2   IMG_38.jpg  /Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DI...   

                                       URL_Publica  
0     https://drive.google.com/uc?id=198ABC98XYZ98  
1  https://drive.google.com/uc?id=1190ABC19

In [3]:
print("🗺️  MAPEO REAL DE GOOGLE DRIVE IDs")
print("=" * 50)

# PASO 1: Crear un diccionario con el mapeo real
# TÚ DEBES LLENAR ESTE DICCIONARIO con los IDs reales de Google Drive

# Ejemplo basado en tu URL: https://drive.google.com/file/d/145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8/view?usp=sharing
# El ID es: 145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8

google_drive_mapping = {
    # Formato: "número_imagen": "ID_de_google_drive"
    # Ejemplo para la primera imagen:
    "98": "145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8",  # ← Tu primer ejemplo
    
    # AÑADE AQUÍ EL RESTO DE TUS IMÁGENES:
    # "190": "ID_de_IMG_190_en_google_drive",
    # "38": "ID_de_IMG_38_en_google_drive", 
    # "73": "ID_de_IMG_73_en_google_drive",
    # "44": "ID_de_IMG_44_en_google_drive",
    # "95": "ID_de_IMG_95_en_google_drive",
    # "42": "ID_de_IMG_42_en_google_drive",
    # "59": "ID_de_IMG_59_en_google_drive",
    # "112": "ID_de_IMG_112_en_google_drive",
    # ... continúa con todas tus imágenes
}

print(f"📋 Mapeo actual definido para {len(google_drive_mapping)} imágenes:")
for num, drive_id in google_drive_mapping.items():
    print(f"   IMG_{num}.jpg → {drive_id}")

print(f"\n🔍 TODAS LAS IMÁGENES EN TU CSV:")
# Cargar CSV para ver todas las imágenes que necesitas mapear
df = pd.read_csv("/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv")

# Extraer todos los números de imagen únicos
numeros_imagenes = []
for imagen in df['Imagen']:
    match = re.search(r'IMG_(\d+)\.jpg', imagen)
    if match:
        numeros_imagenes.append(match.group(1))

numeros_unicos = sorted(set(numeros_imagenes))
print(f"Total de imágenes únicas: {len(numeros_unicos)}")
print(f"Números: {numeros_unicos}")

print(f"\n📝 PARA COMPLETAR EL MAPEO:")
print(f"1. Sube cada una de estas imágenes a Google Drive:")
for num in numeros_unicos[:10]:  # Mostrar solo las primeras 10
    status = "✅ MAPEADA" if num in google_drive_mapping else "❌ FALTA"
    print(f"   IMG_{num}.jpg → {status}")

if len(numeros_unicos) > 10:
    faltantes = len([n for n in numeros_unicos if n not in google_drive_mapping])
    print(f"   ... y {len(numeros_unicos)-10} más ({faltantes} sin mapear)")

print(f"\n📖 PROCESO PARA OBTENER CADA ID:")
print(f"1. Ve a drive.google.com")
print(f"2. Sube IMG_[numero].jpg")
print(f"3. Clic derecho → Obtener enlace")
print(f"4. Cambia a 'Cualquier persona con el enlace'")
print(f"5. Copia el enlace: https://drive.google.com/file/d/[ID]/view?usp=sharing")
print(f"6. Extrae solo el [ID] y añádelo al diccionario google_drive_mapping")

print(f"\n🚀 CUANDO TENGAS TODOS LOS IDs, EJECUTA LA SIGUIENTE CELDA")

🗺️  MAPEO REAL DE GOOGLE DRIVE IDs
📋 Mapeo actual definido para 1 imágenes:
   IMG_98.jpg → 145NoCNASpKY6vY8OCEpcCXSV0V3IO5D8

🔍 TODAS LAS IMÁGENES EN TU CSV:
Total de imágenes únicas: 200
Números: ['1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '21', '22', '23', 

In [4]:
print("🚀 APLICANDO MAPEO REAL Y CREANDO CSV FINAL")
print("=" * 55)

# Cargar el CSV
df = pd.read_csv("/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_ok.csv")

def crear_url_real_google_drive(imagen_name):
    """
    Usa el mapeo real de Google Drive para crear URLs directas
    """
    # Extraer número de la imagen
    match = re.search(r'IMG_(\d+)\.jpg', imagen_name)
    if match:
        numero = match.group(1)
        
        # Buscar en el mapeo real
        if numero in google_drive_mapping:
            google_drive_id = google_drive_mapping[numero]
            url = f"https://drive.google.com/uc?id={google_drive_id}"
            return url
        else:
            print(f"⚠️  No se encontró mapeo para IMG_{numero}.jpg")
            return None
    return None

# Aplicar el mapeo real
print("🔗 Creando URLs reales de Google Drive...")
df['URL_Publica'] = df['Imagen'].apply(crear_url_real_google_drive)

# Verificar resultados
urls_creadas = df['URL_Publica'].notna().sum()
urls_faltantes = df['URL_Publica'].isna().sum()

print(f"✅ URLs creadas exitosamente: {urls_creadas}")
print(f"❌ URLs faltantes (sin mapeo): {urls_faltantes}")

if urls_faltantes > 0:
    print(f"\n🔍 Imágenes sin mapeo:")
    sin_mapeo = df[df['URL_Publica'].isna()]['Imagen'].tolist()
    for img in sin_mapeo[:5]:  # Mostrar solo las primeras 5
        num = re.search(r'IMG_(\d+)\.jpg', img).group(1) if re.search(r'IMG_(\d+)\.jpg', img) else "?"
        print(f"   {img} (número: {num})")
    if len(sin_mapeo) > 5:
        print(f"   ... y {len(sin_mapeo)-5} más")

# Mostrar ejemplos del resultado
print(f"\n📋 RESULTADO FINAL:")
cols_to_show = ['id_post', 'Imagen', 'URL_Publica']
print(df[cols_to_show].head())

# Verificar que las URLs están bien formadas
if urls_creadas > 0:
    url_ejemplo = df['URL_Publica'].dropna().iloc[0]
    print(f"\n🔍 Ejemplo de URL generada:")
    print(f"   {url_ejemplo}")
    
    # Verificar formato
    if url_ejemplo.startswith('https://drive.google.com/uc?id='):
        print(f"   ✅ Formato correcto")
    else:
        print(f"   ❌ Formato incorrecto")

# Guardar CSV final
output_path = "/Users/n.arcos89/Desktop/Bootcamp_Data/DIVA_DIGITAL_Proyecto Final/Data/publicaciones_pixabay_urls_final.csv"
df.to_csv(output_path, index=False)
print(f"\n💾 Archivo guardado en: {output_path}")

# También guardar en la carpeta del proyecto GitHub
github_path = "/Users/n.arcos89/Documents/GitHub/Diva_digital/Data/publicaciones_pixabay_ok.csv"
df.to_csv(github_path, index=False)
print(f"💾 También guardado en GitHub: {github_path}")

print(f"\n📊 RESUMEN:")
print(f"- Total de registros: {len(df)}")
print(f"- URLs de Google Drive creadas: {urls_creadas}")
print(f"- URLs faltantes: {urls_faltantes}")
print(f"- Porcentaje completado: {urls_creadas/len(df)*100:.1f}%")

if urls_creadas == len(df):
    print(f"\n🎉 ¡PERFECTO! Todas las imágenes tienen URLs de Google Drive")
    print(f"📱 Tu app ahora puede usar las imágenes reales desde la nube")
else:
    print(f"\n⚠️  Completa el mapeo para las {urls_faltantes} imágenes restantes")
    print(f"📝 Edita el diccionario google_drive_mapping en la celda anterior")

🚀 APLICANDO MAPEO REAL Y CREANDO CSV FINAL
🔗 Creando URLs reales de Google Drive...
⚠️  No se encontró mapeo para IMG_190.jpg
⚠️  No se encontró mapeo para IMG_38.jpg
⚠️  No se encontró mapeo para IMG_73.jpg
⚠️  No se encontró mapeo para IMG_44.jpg
⚠️  No se encontró mapeo para IMG_95.jpg
⚠️  No se encontró mapeo para IMG_42.jpg
⚠️  No se encontró mapeo para IMG_59.jpg
⚠️  No se encontró mapeo para IMG_112.jpg
⚠️  No se encontró mapeo para IMG_17.jpg
⚠️  No se encontró mapeo para IMG_168.jpg
⚠️  No se encontró mapeo para IMG_193.jpg
⚠️  No se encontró mapeo para IMG_122.jpg
⚠️  No se encontró mapeo para IMG_196.jpg
⚠️  No se encontró mapeo para IMG_90.jpg
⚠️  No se encontró mapeo para IMG_191.jpg
⚠️  No se encontró mapeo para IMG_133.jpg
⚠️  No se encontró mapeo para IMG_141.jpg
⚠️  No se encontró mapeo para IMG_115.jpg
⚠️  No se encontró mapeo para IMG_165.jpg
⚠️  No se encontró mapeo para IMG_171.jpg
⚠️  No se encontró mapeo para IMG_103.jpg
⚠️  No se encontró mapeo para IMG_175.jpg
